# Model Selection

In [1]:
import pandas as pd
import numpy as np
import glob, sys, os
sys.path.append('..')

In [2]:
from modules.plotting_metrics import PlotMetric
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style='white', context='talk', font_scale=0.8)

In [59]:
file_name = './df_DkSc_results_COCRYS_DEKOIS_DUD.pkl'
X_merged_dksc = pd.read_pickle(file_name)
# Extract activity column
y_true_merged = X_merged_dksc['activity']
# Drop column from merged_dkksc
X_merged_dksc = X_merged_dksc.drop('activity', axis=1)
X_merged_dksc.shape

(6233, 136)

### Timeout Decorator

In [6]:
import signal
from functools import wraps

def timeout(n_seconds=300):
    '''Stops a function execution after n seconds'''
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            # Set alarm for n seconds
            signal.alarm(n_seconds)
            try:
                # Call decorated func
                return func(*args, **kwargs)
#             except TimeoutError as e:
                print(f'Execution finished after {n_seconds}:', e)
            finally:
                # Cancel Alarm
                signal.alarm(0)
        return wrapper
    return decorator

## Scaffold Splitting

In [7]:
#*************************************************
# Functions to compute stratify scaffold splitting
#*************************************************
from scaffold_splitter import train_test_scaffold_split

In [8]:
# Compute or load the dataframe containing the Generic Murcko Scaffolds
file = './df_COCRYS_DUD_DEKOIS_Murcko_Scaffolds_SMILES.obj'

df_scff_murcko = pd.read_pickle(file)

## Train/test on the same dataset 

### Learning Curves

In [9]:
from sklearn.model_selection import GridSearchCV, train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from sklearn.pipeline import Pipeline
from sklearn.model_selection import learning_curve

In [10]:
def plot_learning_curves(estimator, X, y, title, ylim=[0.5,1], axes=None,
                         cv=3, train_sizes=np.linspace(0.1, 1.0, 10), 
                         scoring='roc_auc', n_jobs=4):
    '''
    Plot estimator performance on the training and validation
    sets as a function of the training set size.

    Parameters
    ----------
    estimator: sklearn estimator object type
       Object type that implements the "fit" and "predict method"

    X: array-like, shape (m_samples, n_features)
        Training array with m_samples and n_features.
    y: array-like, shape (m_samples)
        Target array relative to X with m labels.
    axes: array of 3 axes
        matplotlib axes array to append the generated plot
    ylim: array
       
    '''
    if axes == None:
        _, axes = plt.subplots(1, 1, figsize=(5, 5))
        
    axes.set(title=title, ylim=ylim, xlabel='Training examples', ylabel=f'Metric: {scoring}')
    # Use learning_curve function from sklearn
    train_sizes, train_scores, test_scores, fit_times, _ = \
       learning_curve(estimator, X, y, scoring=scoring, return_times = True,
                      cv=cv, n_jobs=n_jobs, train_sizes=train_sizes)
    # Compute useful metrics
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
#     fit_times_mean = np.mean(fit_times, axis=1)
#     fit_times_std = np.std(fit_times, axis=1)
    # Plot the learning curve
    axes.grid()
    axes.fill_between(train_sizes, train_scores_mean - train_scores_std,
                        train_scores_mean + train_scores_std, alpha=0.1, color='r')
    axes.fill_between(train_sizes, test_scores_mean - test_scores_std,
                        test_scores_mean + test_scores_std, alpha=0.1, color='g')
    axes.plot(train_sizes, train_scores_mean, 'o-', color='r',
                label='Train score')
    axes.plot(train_sizes, test_scores_mean, 'o-', color='g',
                label='Cross-validation score')
    axes.legend(loc='lower right')
    

### Hyperparameters tunning: Grid Search

In [74]:
def run_grid_search(estimator, X_train, y_train, X_test, y_test, hyperparams,  cv_value=5, 
                    scoring='roc_auc', splitting='random', std_scale=False):

    # Format the hyperparms
    hyperparams_dict = {'estimator__' + key: val for key, val in hyperparams.items()}
    
    if std_scale:
        # Create the Pipe
        scaler = StandardScaler()
        pipe = Pipeline([('scaler', scaler),
                         ('estimator', estimator)])
    else:
        pipe = estimator
    
    # Do Grid Search
    gs = GridSearchCV(estimator = pipe, param_grid = hyperparams,
                     cv = cv_value, scoring = scoring, n_jobs = 6, refit = True)

    # Train the model
    gs.fit(X_train, y_train) 

    # Predictions
    y_train_predict = gs.predict_proba(X_train)
    y_test_predict = gs.predict_proba(X_test)
    
    # Values to return
    n_train_mols    = y_train.shape[0]
    n_train_actives = y_train.sum()
    n_test_mols     = y_test.shape[0]
    n_test_actives  = y_test.sum()
    mean_cv_roc     = gs.best_score_
    train_roc       = roc_auc_score(y_train, y_train_predict[:, 1])
    test_roc        = roc_auc_score(y_test, y_test_predict[:, 1])
    best_params     = gs.best_params_

    # Print some values
    print(f'No. of molecules in train set: {n_train_mols}, with {n_train_actives} actives.')
    print(f'No. of molecules in test set: {n_test_mols}, with {n_test_actives} actives.')
    print('')
    print('*'*10, 'GRID SEARCH RESULTS', '*'*10)
    print('- Mean CV ROC-AUC:\t{:.3f}'.format(mean_cv_roc))
    print('- Train ROC-AUC:\t{:.3f}'.format(train_roc))
    print('- Test ROC-AUC:\t{:.3f}'.format(test_roc))
    print('- Best hyperparameters', best_params)
    print('**'*21)
    print('')
    
    return [n_train_mols, n_train_actives, n_test_mols, n_test_actives,
            mean_cv_roc, train_roc, test_roc, best_params]

In [12]:
def run_cross_val_bias_var_tradeoff(estimator, X_train, y_train, X_test, y_test, cv_value=5, scoring='roc_auc',
                                   score_type='Dk_sc', pred_prob = True):
    
    # Print some values
    print(f'No. of molecules in Train set: {y_train.shape[0]}, with {y_train.sum()} actives.')
    print(f'No. of molecules in Test set: {y_test.shape[0]}, with {y_test.sum()} actives.')

    # Scaler
    scaler = StandardScaler()

    # Set the pipe
    pipe = Pipeline([('scaler', scaler), ('estimator', estimator)])

    # Set the CV score
    roc_cv = cross_val_score(pipe, X_train, y_train, cv=cv_value, scoring=scoring, n_jobs= -1)

    pipe.fit(X_train, y_train)
    
    if pred_prob:
        # Predict proba
        y_predict_train = pipe.predict_proba(X_train)
        y_predict_test = pipe.predict_proba(X_test)
    else:
        y_predict_train = pipe.predict(X_train)
        y_predict_test = pipe.predict(X_test)

    # CV Mean ROC-AUC
    print('CV={} ROC-AUC: {:.4f}'.format(cv_value, roc_cv.mean()))

    # Training ROC-AUC:
    print('Train ROC-AUC: {:.4f}'.format(roc_auc_score(y_train, y_predict_train[:, 1])))

    # Test ROC-AUC:
    print('Test ROC-AUC: {:.4f}'.format(roc_auc_score(y_test, y_predict_test[:, 1])))

### Function to report the Best Conformation's ROC-AUC for a given subset of samples

In [83]:
#************************************************************************
# Returns the best conformatio's ROC-AUC value of a given subset X and y
#************************************************************************

def get_roc_auc_DkSc(X_train, y_train, X_test, y_test, verbose=True):
    roc_auc_train = X_train.apply(
        lambda x: roc_auc_score(y_true= y_train, y_score= -x), axis=0)
    roc_auc_test = X_test.apply(
        lambda x: roc_auc_score(y_true= y_test, y_score= -x), axis=0)
    # Values to return
    train_best_roc = roc_auc_train.max()
    train_median   = roc_auc_train.median()
    train_mean     = roc_auc_train.mean()
    test_best_roc = roc_auc_test.max()
    test_median   = roc_auc_test.median()
    test_mean     = roc_auc_test.mean()
    
    if verbose:
        print("***** Best Conformation's ROC-AUC using docking scores *****")
        
        print("> Train best conf. ROC-AUC: {:.3f}".format(train_best_roc) +
              " \t> median: {:.3f}".format(train_median) +
              ', mean: {:.3f}'.format(train_mean))
        
        print("> Test best conf. ROC-AUC: {:.3f}".format(train_best_roc) +
              " \t> median: {:.3f}".format(train_median) +
              ', mean: {:.3f}'.format(train_mean))
        print('**'*32)
    # return a list of results to capture by the  wrapper function
    return [train_best_roc, train_median, train_mean,
            test_best_roc, test_median, test_mean]
        

In [89]:
#******************************************
# Decorator functions to capture GS results
#******************************************
from functools import wraps

def capture_GS_results(results_dict=None, capture=True):
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            if results_dict != None and capture:
                results = func(*args, **kwargs)
                # Create a key with the first four values
                key = '_'.join(results[:4])
                # Append results to the results dictionary
                results_dict[key] = results
            else:
                return func(*args, **kwargs)
        return wrapper
    return decorator
            
#******************************************************
# Dictionary to capture GS results using the decorator 
#******************************************************
results_dict = {}

#******************************************************
# Function to Split and run Grid Search
#******************************************************
@capture_GS_results(results_dict)
def split_and_gs(train_name, test_name, estimator_name,
                 X, y, estimator, hyperparams, splitting='random', 
                 test_size=0.25, scaffold_series=None, **kwargs):
    '''Given a X and y sets, a sklean estimator and an splitting method, 
    performs Grid Search CV using the parsed hyperparams'''
    #**************
    # Do the split
    #**************
    if splitting == 'scaffold':
        X_train, X_test, y_train, y_test = \
            train_test_scaffold_split(X, y, scaffold_series = scaffold_series,
                test_size=test_size, stratify=y)
    elif splitting == 'random':
        X_train, X_test, y_train, y_test = \
            train_test_split(X, y, test_size=test_size, stratify=y)
        
    # Verbose
    print(f'{estimator_name} => Train: {train_name}; Test: {test_name}; split: {splitting}')
    
    # Function to run Grid Search
    #----------------------------
    gs_results = run_grid_search(estimator, 
                    X_train, y_train, X_test, y_test, 
                    hyperparams = hyperparams,  **kwargs)

    # Function to extract ROC results from DkSc values 
    #-------------------------------------------------
    dksc_results = get_roc_auc_DkSc(X_train, y_train, X_test, y_test)
    
    # Return both list of results to captured by the decorador function
    return [train_name, test_name, estimator_name, splitting] + gs_results + dksc_results

#  Hyperparameter Tunning: Grid Search
<h2 style='background-color: #F9E5AB; padding: 5px;'>
    Train-Test with DEKOIS Library
</h2>
<div style='background-color: #FE8460; min-height: 5px'></div>

In [14]:
#### library = 'DEKOIS'
library = 'DEKOIS'

# Train and test over DEKOIS
X = X_merged_dksc.loc[library]
y = y_true_merged.loc[library]
scaffold_series = df_scff_murcko['scff_generic'].loc[library]

<h3 style='color: #F84122; padding: 0px;; margin: 0px'>GS: Linear SVM </h3>
<b>DEKOIS  - Random and Stratified Scaffold Splitting</b>

In [90]:
%%time
from sklearn.svm import SVC
estimator_name = 'LinearSVC'
estimator = SVC(kernel = 'linear', probability=True)
hyperparams = {'C': np.geomspace(1e-15, 1e-6, 4)}

# RANDOM Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams, 
             splitting='random', test_size=0.25, 
             scaffold_series=None)

LinearSVC => Train: DEKOIS; Test: DEKOIS; split: random
No. of molecules in train set: 4674, with 225 actives.
No. of molecules in test set: 1559, with 75 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.576
- Train ROC-AUC:	0.675
- Test ROC-AUC:	0.674
- Best hyperparameters {'C': 1e-15}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.643 	> median: 0.589, mean: 0.588
> Test best conf. ROC-AUC: 0.643 	> median: 0.589, mean: 0.588
****************************************************************
CPU times: user 2.84 s, sys: 306 ms, total: 3.15 s
Wall time: 22.7 s


In [16]:
%%time
# from sklearn.svm import SVC
estimator = SVC(kernel = 'linear', probability=True)
hyperparams = {'C': np.geomspace(1e-15, 1e-6, 4)}

# SCAFFOLD Train test splitting
split_and_gs(X, y, estimator, hyperparams, 
             splitting='scaffold', test_size=0.25, 
             scaffold_series=scaffold_series)

No. of molecules in train set: 885, with 30 actives.
No. of molecules in test set: 336, with 10 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC score: 0.847
- Train ROC-AUC: 0.875
- Test ROC-AUC: 0.956
- Best hyperparameters {'C': 1e-15}
******************************************

------------------------------------------------------------
**** Best Conformation's ROC-AUC using docking scores ****
> Train best conf. ROC-AUC: 0.884 	> Median ROC-AUC: 0.816
> Test best conf. ROC-AUC: 0.965 	> Median ROC-AUC: 0.897
------------------------------------------------------------
CPU times: user 639 ms, sys: 0 ns, total: 639 ms
Wall time: 751 ms


<h3 style='color: #F84122; padding: 0px;; margin: 0px'>GS: Radial Basis Function SVM </h3>
<b>DEKOIS  - Random and Stratified Scaffold Splitting</b>

In [17]:
%%time
estimator = SVC(kernel = 'rbf', probability=True)

hyperparams = {'C': np.geomspace(1e-8, 1e-2, 4), 
               'gamma': np.geomspace(1e-6, 1e0, 4)}

# RANDOM Train test splitting
split_and_gs(X, y, estimator, hyperparams, 
             splitting='random', test_size=0.25, 
             scaffold_series=None)

No. of molecules in train set: 915, with 30 actives.
No. of molecules in test set: 306, with 10 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC score: 0.879
- Train ROC-AUC: 0.886
- Test ROC-AUC: 0.932
- Best hyperparameters {'C': 1e-08, 'gamma': 0.0001}
******************************************

------------------------------------------------------------
**** Best Conformation's ROC-AUC using docking scores ****
> Train best conf. ROC-AUC: 0.89 	> Median ROC-AUC: 0.819
> Test best conf. ROC-AUC: 0.958 	> Median ROC-AUC: 0.887
------------------------------------------------------------
CPU times: user 980 ms, sys: 0 ns, total: 980 ms
Wall time: 2.23 s


In [18]:
%%time
estimator = SVC(kernel = 'rbf', probability=True)

hyperparams = {'C': np.geomspace(1e-8, 1e-2, 4), 
               'gamma': np.geomspace(1e-6, 1e0, 4)}

# SCAFFOLD Train test splitting
split_and_gs(X, y, estimator, hyperparams, 
             splitting='scaffold', test_size=0.25, 
             scaffold_series=scaffold_series)

No. of molecules in train set: 885, with 30 actives.
No. of molecules in test set: 336, with 10 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC score: 0.846
- Train ROC-AUC: 0.875
- Test ROC-AUC: 0.953
- Best hyperparameters {'C': 1e-08, 'gamma': 1e-06}
******************************************

------------------------------------------------------------
**** Best Conformation's ROC-AUC using docking scores ****
> Train best conf. ROC-AUC: 0.884 	> Median ROC-AUC: 0.816
> Test best conf. ROC-AUC: 0.965 	> Median ROC-AUC: 0.897
------------------------------------------------------------
CPU times: user 979 ms, sys: 0 ns, total: 979 ms
Wall time: 2.18 s


<h3 style='color: #F84122; padding: 0px;; margin: 0px'> GS: Logistic Regression </h3>
<b>DEKOIS  - Random and Stratified Scaffold Splitting</b>

In [19]:
%%time
from sklearn.linear_model import LogisticRegression
estimator = LogisticRegression()

hyperparams = {'C': [0.00001, 0.0001, 0.001, 0.01, 0.1], 
               'penalty': ['l1', 'l2'], 
               'solver': ['lbfgs', 'liblinear']}

# RANDOM Train test splitting
split_and_gs(X, y, estimator, hyperparams, 
             splitting='random', test_size=0.25, 
             scaffold_series=None)

No. of molecules in train set: 915, with 30 actives.
No. of molecules in test set: 306, with 10 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC score: 0.873
- Train ROC-AUC: 0.878
- Test ROC-AUC: 0.951
- Best hyperparameters {'C': 0.001, 'penalty': 'l2', 'solver': 'lbfgs'}
******************************************

------------------------------------------------------------
**** Best Conformation's ROC-AUC using docking scores ****
> Train best conf. ROC-AUC: 0.885 	> Median ROC-AUC: 0.814
> Test best conf. ROC-AUC: 0.964 	> Median ROC-AUC: 0.898
------------------------------------------------------------
CPU times: user 1.48 s, sys: 242 ms, total: 1.72 s
Wall time: 1.35 s


In [20]:
%%time
# from sklearn.linear_model import LogisticRegression
estimator = LogisticRegression()

hyperparams = {'C': [0.00001, 0.0001, 0.001, 0.01, 0.1], 
               'penalty': ['l1', 'l2'], 
               'solver': ['lbfgs', 'liblinear']}

# SCAFFOLD Train test splitting
split_and_gs(X, y, estimator, hyperparams, 
             splitting='scaffold', test_size=0.25, 
             scaffold_series=scaffold_series)

No. of molecules in train set: 885, with 30 actives.
No. of molecules in test set: 336, with 10 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC score: 0.864
- Train ROC-AUC: 0.876
- Test ROC-AUC: 0.956
- Best hyperparameters {'C': 0.001, 'penalty': 'l2', 'solver': 'lbfgs'}
******************************************

------------------------------------------------------------
**** Best Conformation's ROC-AUC using docking scores ****
> Train best conf. ROC-AUC: 0.884 	> Median ROC-AUC: 0.816
> Test best conf. ROC-AUC: 0.965 	> Median ROC-AUC: 0.897
------------------------------------------------------------
CPU times: user 1.18 s, sys: 505 ms, total: 1.69 s
Wall time: 1.26 s


<h3 style='color: #F84122; padding: 0px;; margin: 0px'>GS: K-Neighbors Classifier</h3>
<b>DEKOIS  - Random and Stratified Scaffold Splitting</b>

In [21]:
%%time
from sklearn.neighbors import KNeighborsClassifier 
estimator = KNeighborsClassifier()

hyperparams = {'n_neighbors': [25, 55, 125, 225], 
               'p': [1, 2]
             }

# RANDOM Train test splitting
split_and_gs(X, y, estimator, hyperparams, 
             splitting='random', test_size=0.25, 
             scaffold_series=None)

No. of molecules in train set: 915, with 30 actives.
No. of molecules in test set: 306, with 10 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC score: 0.890
- Train ROC-AUC: 0.904
- Test ROC-AUC: 0.848
- Best hyperparameters {'n_neighbors': 225, 'p': 1}
******************************************

------------------------------------------------------------
**** Best Conformation's ROC-AUC using docking scores ****
> Train best conf. ROC-AUC: 0.918 	> Median ROC-AUC: 0.842
> Test best conf. ROC-AUC: 0.898 	> Median ROC-AUC: 0.805
------------------------------------------------------------
CPU times: user 1.13 s, sys: 159 ms, total: 1.29 s
Wall time: 2.05 s


In [22]:
%%time
from sklearn.neighbors import KNeighborsClassifier 
estimator = KNeighborsClassifier()

hyperparams = {'n_neighbors': [25, 55, 125, 225], 
               'p': [1, 2]
             }

# SCAFFOLD Train test splitting
split_and_gs(X, y, estimator, hyperparams, 
             splitting='scaffold', test_size=0.25, 
             scaffold_series=scaffold_series)

No. of molecules in train set: 885, with 30 actives.
No. of molecules in test set: 336, with 10 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC score: 0.839
- Train ROC-AUC: 0.871
- Test ROC-AUC: 0.953
- Best hyperparameters {'n_neighbors': 225, 'p': 2}
******************************************

------------------------------------------------------------
**** Best Conformation's ROC-AUC using docking scores ****
> Train best conf. ROC-AUC: 0.884 	> Median ROC-AUC: 0.816
> Test best conf. ROC-AUC: 0.965 	> Median ROC-AUC: 0.897
------------------------------------------------------------
CPU times: user 967 ms, sys: 0 ns, total: 967 ms
Wall time: 1.3 s


<h3 style='color: #F84122; padding: 0px;; margin: 0px'>GS: Decision Tree Classifier</h3>
<b>DEKOIS  - Random and Stratified Scaffold Splitting</b>

In [23]:
%%time
from sklearn.tree import DecisionTreeClassifier

estimator = DecisionTreeClassifier(splitter='best')
hyperparams = {'criterion': ['gini', 'entropy'], 
               'max_depth': [2, 3, 5],
               'min_samples_split': [0.2, 0.25, 0.3],
               'max_features': [None, 'sqrt', 'log2']}

# RANDOM Train test splitting
split_and_gs(X, y, estimator, hyperparams, 
             splitting='random', test_size=0.25, 
             scaffold_series=None)

No. of molecules in train set: 915, with 30 actives.
No. of molecules in test set: 306, with 10 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC score: 0.890
- Train ROC-AUC: 0.924
- Test ROC-AUC: 0.657
- Best hyperparameters {'criterion': 'entropy', 'max_depth': 5, 'max_features': 'sqrt', 'min_samples_split': 0.3}
******************************************

------------------------------------------------------------
**** Best Conformation's ROC-AUC using docking scores ****
> Train best conf. ROC-AUC: 0.937 	> Median ROC-AUC: 0.878
> Test best conf. ROC-AUC: 0.833 	> Median ROC-AUC: 0.709
------------------------------------------------------------
CPU times: user 1.53 s, sys: 0 ns, total: 1.53 s
Wall time: 1.89 s


In [24]:
%%time
from sklearn.tree import DecisionTreeClassifier

estimator = DecisionTreeClassifier(splitter='best')
hyperparams = {'criterion': ['gini', 'entropy'], 
               'max_depth': [2, 3, 5],
               'min_samples_split': [0.2, 0.25, 0.3],
               'max_features': [None, 'sqrt', 'log2']}

# SCAFFOLD Train test splitting
split_and_gs(X, y, estimator, hyperparams, 
             splitting='scaffold', test_size=0.25, 
             scaffold_series=scaffold_series)

No. of molecules in train set: 885, with 30 actives.
No. of molecules in test set: 336, with 10 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC score: 0.817
- Train ROC-AUC: 0.838
- Test ROC-AUC: 0.746
- Best hyperparameters {'criterion': 'gini', 'max_depth': 5, 'max_features': 'log2', 'min_samples_split': 0.2}
******************************************

------------------------------------------------------------
**** Best Conformation's ROC-AUC using docking scores ****
> Train best conf. ROC-AUC: 0.884 	> Median ROC-AUC: 0.816
> Test best conf. ROC-AUC: 0.965 	> Median ROC-AUC: 0.897
------------------------------------------------------------
CPU times: user 1.48 s, sys: 150 ms, total: 1.63 s
Wall time: 1.83 s


<h3 style='color: #F84122; padding: 0px;; margin: 0px'>GS: Bagging Classifier (k-NN as base estimator) </h3>
<b>DEKOIS  - Random and Stratified Scaffold Splitting</b>

In [25]:
%%time
from sklearn.ensemble import BaggingClassifier

knn = KNeighborsClassifier(n_neighbors=125, p=1, 
                           weights='distance')

estimator = BaggingClassifier(base_estimator=knn, 
                              n_jobs=6, oob_score=True)

hyperparams = {'n_estimators': [300]}

# RANDOM Train test splitting
# split_and_gs(X, y, estimator, hyperparams, 
#              splitting='random', test_size=0.25, 
#              scaffold_series=None)

CPU times: user 23.7 ms, sys: 0 ns, total: 23.7 ms
Wall time: 240 ms


In [26]:
%%time
# from sklearn.ensemble import BaggingClassifier

knn = KNeighborsClassifier(n_neighbors=125, p=1, 
                           weights='distance')

estimator = BaggingClassifier(base_estimator=knn, 
                              n_jobs=6, oob_score=True)

hyperparams = {'n_estimators': [300]}

# SCAFFOLD Train test splitting
# split_and_gs(X, y, estimator, hyperparams, 
#              splitting='scaffold', test_size=0.25, 
#              scaffold_series=scaffold_series)

CPU times: user 39 µs, sys: 23 µs, total: 62 µs
Wall time: 110 µs


<h3 style='color: #F84122; padding: 0px;; margin: 0px'>GS:Random Forest </h3>
<b>DEKOIS  - Random and Stratified Scaffold Splitting</b>

In [27]:
%%time
from sklearn.ensemble import RandomForestClassifier

estimator = RandomForestClassifier()

hyperparams = {'n_estimators': [300, 500],
             'max_depth': [2,  5],
             'min_samples_leaf': [0.1, 0.2],
             'max_features': ['log2', 'sqrt']
            }

# RANDOM Train test splitting
split_and_gs(X, y, estimator, hyperparams, 
             splitting='random', test_size=0.25, 
             scaffold_series=None)

No. of molecules in train set: 915, with 30 actives.
No. of molecules in test set: 306, with 10 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC score: 0.879
- Train ROC-AUC: 0.927
- Test ROC-AUC: 0.868
- Best hyperparameters {'max_depth': 5, 'max_features': 'sqrt', 'min_samples_leaf': 0.1, 'n_estimators': 300}
******************************************

------------------------------------------------------------
**** Best Conformation's ROC-AUC using docking scores ****
> Train best conf. ROC-AUC: 0.896 	> Median ROC-AUC: 0.821
> Test best conf. ROC-AUC: 0.952 	> Median ROC-AUC: 0.884
------------------------------------------------------------
CPU times: user 1.64 s, sys: 170 ms, total: 1.81 s
Wall time: 21.5 s


In [28]:
%%time
from sklearn.ensemble import RandomForestClassifier

estimator = RandomForestClassifier()

hyperparams = {'n_estimators': [300, 500],
             'max_depth': [2,  5],
             'min_samples_leaf': [0.1, 0.2],
             'max_features': ['log2', 'sqrt']
            }

# SCAFFOLD Train test splitting
split_and_gs(X, y, estimator, hyperparams, 
             splitting='scaffold', test_size=0.25, 
             scaffold_series=scaffold_series)

No. of molecules in train set: 885, with 30 actives.
No. of molecules in test set: 336, with 10 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC score: 0.858
- Train ROC-AUC: 0.908
- Test ROC-AUC: 0.961
- Best hyperparameters {'max_depth': 2, 'max_features': 'sqrt', 'min_samples_leaf': 0.1, 'n_estimators': 300}
******************************************

------------------------------------------------------------
**** Best Conformation's ROC-AUC using docking scores ****
> Train best conf. ROC-AUC: 0.884 	> Median ROC-AUC: 0.816
> Test best conf. ROC-AUC: 0.965 	> Median ROC-AUC: 0.897
------------------------------------------------------------
CPU times: user 1.71 s, sys: 0 ns, total: 1.71 s
Wall time: 17.9 s


***
<h2 style='background-color: #F9E5AB; padding: 5px;'>
    Train-Test with DUD Library
</h2>
<div style='background-color: #FE8460; min-height: 5px'></div>

In [14]:
library = 'DUD'

# Train and test over DUDU
X = X_merged_dksc.loc[library]
y = y_true_merged.loc[library]
scaffold_series = df_scff_murcko['scff_generic'].loc[library]

<h3 style='color: #F84122; padding: 0px;; margin: 0px'>GS: Linear SVM </h3>
<b>DUD  - Random and Stratified Scaffold Splitting</b>

In [15]:
%%time
from sklearn.svm import SVC
estimator = SVC(kernel = 'linear', probability=True)
hyperparams = {'C': np.geomspace(1e-15, 1e-6, 4)}

# RANDOM Train test splitting
split_and_gs(X, y, estimator, hyperparams, 
             splitting='random', test_size=0.25, 
             scaffold_series=None)

No. of molecules in train set: 915, with 30 actives.
No. of molecules in test set: 306, with 10 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC score: 0.864
- Train ROC-AUC: 0.881
- Test ROC-AUC: 0.930
- Best hyperparameters {'C': 1e-12}
******************************************

------------------------------------------------------------
**** Best Conformation's ROC-AUC using docking scores ****
> Train best conf. ROC-AUC: 0.888 	> Median ROC-AUC: 0.813
> Test best conf. ROC-AUC: 0.962 	> Median ROC-AUC: 0.894
------------------------------------------------------------
CPU times: user 603 ms, sys: 302 ms, total: 906 ms
Wall time: 2.55 s


In [16]:
%%time
# from sklearn.svm import SVC
estimator = SVC(kernel = 'linear', probability=True)
hyperparams = {'C': np.geomspace(1e-15, 1e-6, 4)}

# SCAFFOLD Train test splitting
split_and_gs(X, y, estimator, hyperparams, 
             splitting='scaffold', test_size=0.25, 
             scaffold_series=scaffold_series)

No. of molecules in train set: 885, with 30 actives.
No. of molecules in test set: 336, with 10 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC score: 0.847
- Train ROC-AUC: 0.875
- Test ROC-AUC: 0.956
- Best hyperparameters {'C': 1e-15}
******************************************

------------------------------------------------------------
**** Best Conformation's ROC-AUC using docking scores ****
> Train best conf. ROC-AUC: 0.884 	> Median ROC-AUC: 0.816
> Test best conf. ROC-AUC: 0.965 	> Median ROC-AUC: 0.897
------------------------------------------------------------
CPU times: user 639 ms, sys: 0 ns, total: 639 ms
Wall time: 751 ms


<h3 style='color: #F84122; padding: 0px;; margin: 0px'>GS: Radial Basis Function SVM </h3>
<b>DUD  - Random and Stratified Scaffold Splitting</b>

In [17]:
%%time
estimator = SVC(kernel = 'rbf', probability=True)

hyperparams = {'C': np.geomspace(1e-8, 1e-2, 4), 
               'gamma': np.geomspace(1e-6, 1e0, 4)}

# RANDOM Train test splitting
split_and_gs(X, y, estimator, hyperparams, 
             splitting='random', test_size=0.25, 
             scaffold_series=None)

No. of molecules in train set: 915, with 30 actives.
No. of molecules in test set: 306, with 10 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC score: 0.879
- Train ROC-AUC: 0.886
- Test ROC-AUC: 0.932
- Best hyperparameters {'C': 1e-08, 'gamma': 0.0001}
******************************************

------------------------------------------------------------
**** Best Conformation's ROC-AUC using docking scores ****
> Train best conf. ROC-AUC: 0.89 	> Median ROC-AUC: 0.819
> Test best conf. ROC-AUC: 0.958 	> Median ROC-AUC: 0.887
------------------------------------------------------------
CPU times: user 980 ms, sys: 0 ns, total: 980 ms
Wall time: 2.23 s


In [18]:
%%time
estimator = SVC(kernel = 'rbf', probability=True)

hyperparams = {'C': np.geomspace(1e-8, 1e-2, 4), 
               'gamma': np.geomspace(1e-6, 1e0, 4)}

# SCAFFOLD Train test splitting
split_and_gs(X, y, estimator, hyperparams, 
             splitting='scaffold', test_size=0.25, 
             scaffold_series=scaffold_series)

No. of molecules in train set: 885, with 30 actives.
No. of molecules in test set: 336, with 10 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC score: 0.846
- Train ROC-AUC: 0.875
- Test ROC-AUC: 0.953
- Best hyperparameters {'C': 1e-08, 'gamma': 1e-06}
******************************************

------------------------------------------------------------
**** Best Conformation's ROC-AUC using docking scores ****
> Train best conf. ROC-AUC: 0.884 	> Median ROC-AUC: 0.816
> Test best conf. ROC-AUC: 0.965 	> Median ROC-AUC: 0.897
------------------------------------------------------------
CPU times: user 979 ms, sys: 0 ns, total: 979 ms
Wall time: 2.18 s


<h3 style='color: #F84122; padding: 0px;; margin: 0px'> GS: Logistic Regression </h3>
<b>DUD  - Random and Stratified Scaffold Splitting</b>

In [19]:
%%time
from sklearn.linear_model import LogisticRegression
estimator = LogisticRegression()

hyperparams = {'C': [0.00001, 0.0001, 0.001, 0.01, 0.1], 
               'penalty': ['l1', 'l2'], 
               'solver': ['lbfgs', 'liblinear']}

# RANDOM Train test splitting
split_and_gs(X, y, estimator, hyperparams, 
             splitting='random', test_size=0.25, 
             scaffold_series=None)

No. of molecules in train set: 915, with 30 actives.
No. of molecules in test set: 306, with 10 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC score: 0.873
- Train ROC-AUC: 0.878
- Test ROC-AUC: 0.951
- Best hyperparameters {'C': 0.001, 'penalty': 'l2', 'solver': 'lbfgs'}
******************************************

------------------------------------------------------------
**** Best Conformation's ROC-AUC using docking scores ****
> Train best conf. ROC-AUC: 0.885 	> Median ROC-AUC: 0.814
> Test best conf. ROC-AUC: 0.964 	> Median ROC-AUC: 0.898
------------------------------------------------------------
CPU times: user 1.48 s, sys: 242 ms, total: 1.72 s
Wall time: 1.35 s


In [20]:
%%time
# from sklearn.linear_model import LogisticRegression
estimator = LogisticRegression()

hyperparams = {'C': [0.00001, 0.0001, 0.001, 0.01, 0.1], 
               'penalty': ['l1', 'l2'], 
               'solver': ['lbfgs', 'liblinear']}

# SCAFFOLD Train test splitting
split_and_gs(X, y, estimator, hyperparams, 
             splitting='scaffold', test_size=0.25, 
             scaffold_series=scaffold_series)

No. of molecules in train set: 885, with 30 actives.
No. of molecules in test set: 336, with 10 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC score: 0.864
- Train ROC-AUC: 0.876
- Test ROC-AUC: 0.956
- Best hyperparameters {'C': 0.001, 'penalty': 'l2', 'solver': 'lbfgs'}
******************************************

------------------------------------------------------------
**** Best Conformation's ROC-AUC using docking scores ****
> Train best conf. ROC-AUC: 0.884 	> Median ROC-AUC: 0.816
> Test best conf. ROC-AUC: 0.965 	> Median ROC-AUC: 0.897
------------------------------------------------------------
CPU times: user 1.18 s, sys: 505 ms, total: 1.69 s
Wall time: 1.26 s


<h3 style='color: #F84122; padding: 0px;; margin: 0px'>GS: K-Neighbors Classifier</h3>
<b>DUD  - Random and Stratified Scaffold Splitting</b>

In [21]:
%%time
from sklearn.neighbors import KNeighborsClassifier 
estimator = KNeighborsClassifier()

hyperparams = {'n_neighbors': [25, 55, 125, 225], 
               'p': [1, 2]
             }

# RANDOM Train test splitting
split_and_gs(X, y, estimator, hyperparams, 
             splitting='random', test_size=0.25, 
             scaffold_series=None)

No. of molecules in train set: 915, with 30 actives.
No. of molecules in test set: 306, with 10 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC score: 0.890
- Train ROC-AUC: 0.904
- Test ROC-AUC: 0.848
- Best hyperparameters {'n_neighbors': 225, 'p': 1}
******************************************

------------------------------------------------------------
**** Best Conformation's ROC-AUC using docking scores ****
> Train best conf. ROC-AUC: 0.918 	> Median ROC-AUC: 0.842
> Test best conf. ROC-AUC: 0.898 	> Median ROC-AUC: 0.805
------------------------------------------------------------
CPU times: user 1.13 s, sys: 159 ms, total: 1.29 s
Wall time: 2.05 s


In [22]:
%%time
from sklearn.neighbors import KNeighborsClassifier 
estimator = KNeighborsClassifier()

hyperparams = {'n_neighbors': [25, 55, 125, 225], 
               'p': [1, 2]
             }

# SCAFFOLD Train test splitting
split_and_gs(X, y, estimator, hyperparams, 
             splitting='scaffold', test_size=0.25, 
             scaffold_series=scaffold_series)

No. of molecules in train set: 885, with 30 actives.
No. of molecules in test set: 336, with 10 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC score: 0.839
- Train ROC-AUC: 0.871
- Test ROC-AUC: 0.953
- Best hyperparameters {'n_neighbors': 225, 'p': 2}
******************************************

------------------------------------------------------------
**** Best Conformation's ROC-AUC using docking scores ****
> Train best conf. ROC-AUC: 0.884 	> Median ROC-AUC: 0.816
> Test best conf. ROC-AUC: 0.965 	> Median ROC-AUC: 0.897
------------------------------------------------------------
CPU times: user 967 ms, sys: 0 ns, total: 967 ms
Wall time: 1.3 s


<h3 style='color: #F84122; padding: 0px;; margin: 0px'>GS: Decision Tree Classifier</h3>
<b>DUD  - Random and Stratified Scaffold Splitting</b>

In [23]:
%%time
from sklearn.tree import DecisionTreeClassifier

estimator = DecisionTreeClassifier(splitter='best')
hyperparams = {'criterion': ['gini', 'entropy'], 
               'max_depth': [2, 3, 5],
               'min_samples_split': [0.2, 0.25, 0.3],
               'max_features': [None, 'sqrt', 'log2']}

# RANDOM Train test splitting
split_and_gs(X, y, estimator, hyperparams, 
             splitting='random', test_size=0.25, 
             scaffold_series=None)

No. of molecules in train set: 915, with 30 actives.
No. of molecules in test set: 306, with 10 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC score: 0.890
- Train ROC-AUC: 0.924
- Test ROC-AUC: 0.657
- Best hyperparameters {'criterion': 'entropy', 'max_depth': 5, 'max_features': 'sqrt', 'min_samples_split': 0.3}
******************************************

------------------------------------------------------------
**** Best Conformation's ROC-AUC using docking scores ****
> Train best conf. ROC-AUC: 0.937 	> Median ROC-AUC: 0.878
> Test best conf. ROC-AUC: 0.833 	> Median ROC-AUC: 0.709
------------------------------------------------------------
CPU times: user 1.53 s, sys: 0 ns, total: 1.53 s
Wall time: 1.89 s


In [24]:
%%time
from sklearn.tree import DecisionTreeClassifier

estimator = DecisionTreeClassifier(splitter='best')
hyperparams = {'criterion': ['gini', 'entropy'], 
               'max_depth': [2, 3, 5],
               'min_samples_split': [0.2, 0.25, 0.3],
               'max_features': [None, 'sqrt', 'log2']}

# SCAFFOLD Train test splitting
split_and_gs(X, y, estimator, hyperparams, 
             splitting='scaffold', test_size=0.25, 
             scaffold_series=scaffold_series)

No. of molecules in train set: 885, with 30 actives.
No. of molecules in test set: 336, with 10 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC score: 0.817
- Train ROC-AUC: 0.838
- Test ROC-AUC: 0.746
- Best hyperparameters {'criterion': 'gini', 'max_depth': 5, 'max_features': 'log2', 'min_samples_split': 0.2}
******************************************

------------------------------------------------------------
**** Best Conformation's ROC-AUC using docking scores ****
> Train best conf. ROC-AUC: 0.884 	> Median ROC-AUC: 0.816
> Test best conf. ROC-AUC: 0.965 	> Median ROC-AUC: 0.897
------------------------------------------------------------
CPU times: user 1.48 s, sys: 150 ms, total: 1.63 s
Wall time: 1.83 s


<h3 style='color: #F84122; padding: 0px;; margin: 0px'>GS: Bagging Classifier (k-NN as base estimator) </h3>
<b>DUD  - Random and Stratified Scaffold Splitting</b>

In [25]:
%%time
from sklearn.ensemble import BaggingClassifier

knn = KNeighborsClassifier(n_neighbors=125, p=1, 
                           weights='distance')

estimator = BaggingClassifier(base_estimator=knn, 
                              n_jobs=6, oob_score=True)

hyperparams = {'n_estimators': [300]}

# RANDOM Train test splitting
# split_and_gs(X, y, estimator, hyperparams, 
#              splitting='random', test_size=0.25, 
#              scaffold_series=None)

CPU times: user 23.7 ms, sys: 0 ns, total: 23.7 ms
Wall time: 240 ms


In [26]:
%%time
# from sklearn.ensemble import BaggingClassifier

knn = KNeighborsClassifier(n_neighbors=125, p=1, 
                           weights='distance')

estimator = BaggingClassifier(base_estimator=knn, 
                              n_jobs=6, oob_score=True)

hyperparams = {'n_estimators': [300]}

# SCAFFOLD Train test splitting
# split_and_gs(X, y, estimator, hyperparams, 
#              splitting='scaffold', test_size=0.25, 
#              scaffold_series=scaffold_series)

CPU times: user 39 µs, sys: 23 µs, total: 62 µs
Wall time: 110 µs


<h3 style='color: #F84122; padding: 0px;; margin: 0px'>GS:Random Forest </h3>
<b>DUD  - Random and Stratified Scaffold Splitting</b>

In [27]:
%%time
from sklearn.ensemble import RandomForestClassifier

estimator = RandomForestClassifier()

hyperparams = {'n_estimators': [300, 500],
             'max_depth': [2,  5],
             'min_samples_leaf': [0.1, 0.2],
             'max_features': ['log2', 'sqrt']
            }

# RANDOM Train test splitting
split_and_gs(X, y, estimator, hyperparams, 
             splitting='random', test_size=0.25, 
             scaffold_series=None)

No. of molecules in train set: 915, with 30 actives.
No. of molecules in test set: 306, with 10 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC score: 0.879
- Train ROC-AUC: 0.927
- Test ROC-AUC: 0.868
- Best hyperparameters {'max_depth': 5, 'max_features': 'sqrt', 'min_samples_leaf': 0.1, 'n_estimators': 300}
******************************************

------------------------------------------------------------
**** Best Conformation's ROC-AUC using docking scores ****
> Train best conf. ROC-AUC: 0.896 	> Median ROC-AUC: 0.821
> Test best conf. ROC-AUC: 0.952 	> Median ROC-AUC: 0.884
------------------------------------------------------------
CPU times: user 1.64 s, sys: 170 ms, total: 1.81 s
Wall time: 21.5 s


In [28]:
%%time
from sklearn.ensemble import RandomForestClassifier

estimator = RandomForestClassifier()

hyperparams = {'n_estimators': [300, 500],
             'max_depth': [2,  5],
             'min_samples_leaf': [0.1, 0.2],
             'max_features': ['log2', 'sqrt']
            }

# SCAFFOLD Train test splitting
split_and_gs(X, y, estimator, hyperparams, 
             splitting='scaffold', test_size=0.25, 
             scaffold_series=scaffold_series)

No. of molecules in train set: 885, with 30 actives.
No. of molecules in test set: 336, with 10 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC score: 0.858
- Train ROC-AUC: 0.908
- Test ROC-AUC: 0.961
- Best hyperparameters {'max_depth': 2, 'max_features': 'sqrt', 'min_samples_leaf': 0.1, 'n_estimators': 300}
******************************************

------------------------------------------------------------
**** Best Conformation's ROC-AUC using docking scores ****
> Train best conf. ROC-AUC: 0.884 	> Median ROC-AUC: 0.816
> Test best conf. ROC-AUC: 0.965 	> Median ROC-AUC: 0.897
------------------------------------------------------------
CPU times: user 1.71 s, sys: 0 ns, total: 1.71 s
Wall time: 17.9 s


***

<h2 style='background-color: #F9E5AB; padding: 5px;'>
    Train-Test with different libraries 
</h2>
<div style='background-color: #FE8460; min-height: 5px'></div>

##### Here, Random or Scaffold Splitting are not used

In [29]:
# Train DEKOIS, test DUD
def split_by_library_and_gs(X, y, lib_train_name, lib_test_name):
    '''Train-Test "split" by library, and run Grid Search. Splits the main dataframe by library
    using different molecular libraries for Training and Testing.'''
    X_train = X.loc[lib_train_name]
    y_train = y.loc[lib_train_name]

    X_test = X.loc[lib_test_name]
    y_test = y.loc[lib_test_name]

    run_grid_search(estimator, 
                        X_train, y_train, X_test, y_test, 
                        hyperparams = hyperparams)

    _get_best_conf_roc_auc(X_train, y_train, X_test, y_test)

In [30]:
# **********************************************
# Train and Test sets from diferent libraries
# **********************************************
X = X_merged_dksc
y = y_true_merged

<h3 style='color: #F84122; padding: 0px;; margin: 0px'>GS: Linear SVM  </h3>
<b>Train and Test with different libraries</b>

In [31]:
%%time
from sklearn.svm import SVC
estimator = SVC(kernel = 'linear', probability=True)
hyperparams = {'C': np.geomspace(1e-15, 1e-6, 4)}

# Train DEKOIS, test DUD
train_lib = 'DEKOIS'
test_lib = 'DUD'
split_by_library_and_gs(X, y, train_lib, test_lib)

No. of molecules in train set: 1221, with 40 actives.
No. of molecules in test set: 4893, with 141 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC score: 0.889
- Train ROC-AUC: 0.894
- Test ROC-AUC: 0.607
- Best hyperparameters {'C': 1e-15}
******************************************

------------------------------------------------------------
**** Best Conformation's ROC-AUC using docking scores ****
> Train best conf. ROC-AUC: 0.894 	> Median ROC-AUC: 0.833
> Test best conf. ROC-AUC: 0.72 	> Median ROC-AUC: 0.571
------------------------------------------------------------
CPU times: user 760 ms, sys: 0 ns, total: 760 ms
Wall time: 1.18 s


In [32]:
%%time
# from sklearn.svm import SVC
estimator = SVC(kernel = 'linear', probability=True)
hyperparams = {'C': np.geomspace(1e-15, 1e-6, 4)}

# Train DEKOIS, test DUD
train_lib = 'DUD'
test_lib = 'DEKOIS'
split_by_library_and_gs(X, y, train_lib, test_lib)

No. of molecules in train set: 4893, with 141 actives.
No. of molecules in test set: 1221, with 40 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC score: 0.782
- Train ROC-AUC: 0.800
- Test ROC-AUC: 0.870
- Best hyperparameters {'C': 1e-15}
******************************************

------------------------------------------------------------
**** Best Conformation's ROC-AUC using docking scores ****
> Train best conf. ROC-AUC: 0.72 	> Median ROC-AUC: 0.571
> Test best conf. ROC-AUC: 0.894 	> Median ROC-AUC: 0.833
------------------------------------------------------------
CPU times: user 2.15 s, sys: 0 ns, total: 2.15 s
Wall time: 10.9 s


<h3 style='color: #F84122; padding: 0px;; margin: 0px'>GS: Radial Basis Function SVM </h3>
<b>Train and Test with different libraries</b>

In [33]:
%%time
estimator = SVC(kernel = 'rbf', probability=True)

hyperparams = {'C': np.geomspace(1e-8, 1e-2, 4), 
               'gamma': np.geomspace(1e-6, 1e0, 4)}

# Train DEKOIS, test DUD
train_lib = 'DEKOIS'
test_lib = 'DUD'
split_by_library_and_gs(X, y, train_lib, test_lib)

No. of molecules in train set: 1221, with 40 actives.
No. of molecules in test set: 4893, with 141 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC score: 0.888
- Train ROC-AUC: 0.893
- Test ROC-AUC: 0.613
- Best hyperparameters {'C': 1e-08, 'gamma': 1e-06}
******************************************

------------------------------------------------------------
**** Best Conformation's ROC-AUC using docking scores ****
> Train best conf. ROC-AUC: 0.894 	> Median ROC-AUC: 0.833
> Test best conf. ROC-AUC: 0.72 	> Median ROC-AUC: 0.571
------------------------------------------------------------
CPU times: user 1.12 s, sys: 0 ns, total: 1.12 s
Wall time: 3.53 s


In [34]:
%%time
estimator = SVC(kernel = 'rbf', probability=True)

hyperparams = {'C': np.geomspace(1e-8, 1e-2, 4), 
               'gamma': np.geomspace(1e-6, 1e0, 4)}

# Train DEKOIS, test DUD
train_lib = 'DUD'
test_lib = 'DEKOIS'
split_by_library_and_gs(X, y, train_lib, test_lib)

No. of molecules in train set: 4893, with 141 actives.
No. of molecules in test set: 1221, with 40 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC score: 0.867
- Train ROC-AUC: 0.977
- Test ROC-AUC: 0.638
- Best hyperparameters {'C': 0.01, 'gamma': 0.01}
******************************************

------------------------------------------------------------
**** Best Conformation's ROC-AUC using docking scores ****
> Train best conf. ROC-AUC: 0.72 	> Median ROC-AUC: 0.571
> Test best conf. ROC-AUC: 0.894 	> Median ROC-AUC: 0.833
------------------------------------------------------------
CPU times: user 5.61 s, sys: 0 ns, total: 5.61 s
Wall time: 1min 6s


<h3 style='color: #F84122; padding: 0px;; margin: 0px'>GS: Logistic Regression</h3>
<b>Train and Test with different libraries</b>

In [35]:
%%time
from sklearn.linear_model import LogisticRegression
estimator = LogisticRegression()

hyperparams = {'C': np.geomspace(1e-4, 1e5, 4),
               'penalty': ['l1', 'l2'], 
               'solver': ['lbfgs', 'liblinear']}

# Train DEKOIS, test DUD
train_lib = 'DEKOIS'
test_lib = 'DUD'
split_by_library_and_gs(X, y, train_lib, test_lib)

No. of molecules in train set: 1221, with 40 actives.
No. of molecules in test set: 4893, with 141 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC score: 0.891
- Train ROC-AUC: 0.890
- Test ROC-AUC: 0.603
- Best hyperparameters {'C': 0.0001, 'penalty': 'l2', 'solver': 'lbfgs'}
******************************************

------------------------------------------------------------
**** Best Conformation's ROC-AUC using docking scores ****
> Train best conf. ROC-AUC: 0.894 	> Median ROC-AUC: 0.833
> Test best conf. ROC-AUC: 0.72 	> Median ROC-AUC: 0.571
------------------------------------------------------------
CPU times: user 1.66 s, sys: 0 ns, total: 1.66 s
Wall time: 18.7 s


In [36]:
%%time
# from sklearn.linear_model import LogisticRegression
estimator = LogisticRegression()

hyperparams = {'C': np.geomspace(1e-4, 1e5, 4), 
               'penalty': ['l1', 'l2'], 
               'solver': ['lbfgs', 'liblinear']}

# Train DEKOIS, test DUD
train_lib = 'DUD'
test_lib = 'DEKOIS'
split_by_library_and_gs(X, y, train_lib, test_lib)

No. of molecules in train set: 4893, with 141 actives.
No. of molecules in test set: 1221, with 40 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC score: 0.815
- Train ROC-AUC: 0.922
- Test ROC-AUC: 0.758
- Best hyperparameters {'C': 100.0, 'penalty': 'l2', 'solver': 'liblinear'}
******************************************

------------------------------------------------------------
**** Best Conformation's ROC-AUC using docking scores ****
> Train best conf. ROC-AUC: 0.72 	> Median ROC-AUC: 0.571
> Test best conf. ROC-AUC: 0.894 	> Median ROC-AUC: 0.833
------------------------------------------------------------
CPU times: user 2.61 s, sys: 46.6 ms, total: 2.66 s
Wall time: 49 s


<h3 style='color: #F84122; padding: 0px;; margin: 0px'>GS:Random Forest  </h3>
<b>Train and Test with different libraries</b>

In [37]:
%%time
from sklearn.ensemble import RandomForestClassifier

estimator = RandomForestClassifier()

hyperparams = {'n_estimators': [300, 500],
             'max_depth': [3,  5, 7],
             'min_samples_leaf': [0.03, 0.05],
               'min_samples_split': [0.2,  0.3],
             'max_features': ['sqrt']
            }

# Train DEKOIS, test DUD
train_lib = 'DEKOIS'
test_lib = 'DUD'
split_by_library_and_gs(X, y, train_lib, test_lib)

No. of molecules in train set: 1221, with 40 actives.
No. of molecules in test set: 4893, with 141 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC score: 0.878
- Train ROC-AUC: 0.927
- Test ROC-AUC: 0.572
- Best hyperparameters {'max_depth': 3, 'max_features': 'sqrt', 'min_samples_leaf': 0.05, 'min_samples_split': 0.3, 'n_estimators': 300}
******************************************

------------------------------------------------------------
**** Best Conformation's ROC-AUC using docking scores ****
> Train best conf. ROC-AUC: 0.894 	> Median ROC-AUC: 0.833
> Test best conf. ROC-AUC: 0.72 	> Median ROC-AUC: 0.571
------------------------------------------------------------
CPU times: user 2.18 s, sys: 24.8 ms, total: 2.21 s
Wall time: 39.2 s


In [38]:
%%time
from sklearn.ensemble import RandomForestClassifier

estimator = RandomForestClassifier()

hyperparams = {'n_estimators': [300, 500],
             'max_depth': [3,  5, 7],
             'min_samples_leaf': [0.03, 0.05],
               'min_samples_split': [0.2,  0.3],
             'max_features': ['sqrt']
            }

# Train DEKOIS, test DUD
train_lib = 'DUD'
test_lib = 'DEKOIS'
split_by_library_and_gs(X, y, train_lib, test_lib)

No. of molecules in train set: 4893, with 141 actives.
No. of molecules in test set: 1221, with 40 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC score: 0.803
- Train ROC-AUC: 0.866
- Test ROC-AUC: 0.683
- Best hyperparameters {'max_depth': 7, 'max_features': 'sqrt', 'min_samples_leaf': 0.03, 'min_samples_split': 0.2, 'n_estimators': 300}
******************************************

------------------------------------------------------------
**** Best Conformation's ROC-AUC using docking scores ****
> Train best conf. ROC-AUC: 0.72 	> Median ROC-AUC: 0.571
> Test best conf. ROC-AUC: 0.894 	> Median ROC-AUC: 0.833
------------------------------------------------------------
CPU times: user 4.46 s, sys: 0 ns, total: 4.46 s
Wall time: 1min 20s


***
<h2 style='background-color: #F9E5AB; padding: 5px;'>
    Merged libraries 
</h2>
<div style='background-color: #FE8460; min-height: 5px'></div>

#### DEKOIS, DUD and COCRYS are  treated as one unique library

In [63]:
# Train and test over 
X = X_merged_dksc
y = y_true_merged

scaffold_series = df_scff_murcko['scff_generic']

<h3 style='color: #F84122; padding: 0px;; margin: 0px'>GS: Linear SVM </h3>
<b>Merged Libraries</b>

In [40]:
%%time
from sklearn.svm import SVC
estimator = SVC(kernel = 'linear', probability=True)
hyperparams = {'C': np.geomspace(1e-15, 1e-9, 3)}

# RANDOM Train test splitting
split_and_gs(X, y, estimator, hyperparams, 
             splitting='random', test_size=0.25, 
             scaffold_series=None)

No. of molecules in train set: 4674, with 225 actives.
No. of molecules in test set: 1559, with 75 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC score: 0.712
- Train ROC-AUC: 0.783
- Test ROC-AUC: 0.758
- Best hyperparameters {'C': 1e-15}
******************************************

------------------------------------------------------------
**** Best Conformation's ROC-AUC using docking scores ****
> Train best conf. ROC-AUC: 0.683 	> Median ROC-AUC: 0.602
> Test best conf. ROC-AUC: 0.7 	> Median ROC-AUC: 0.632
------------------------------------------------------------
CPU times: user 3.11 s, sys: 0 ns, total: 3.11 s
Wall time: 12.9 s


In [64]:
%%time
# from sklearn.svm import SVC
estimator = SVC(kernel = 'linear', probability=True)
hyperparams = {'C': np.geomspace(1e-15, 1e-9, 3)}

# SCAFFOLD Train test splitting
split_and_gs(X, y, estimator, hyperparams, 
             splitting='scaffold', test_size=0.25, 
             scaffold_series=scaffold_series)

No. of molecules in train set: 4449, with 225 actives.
No. of molecules in test set: 1784, with 75 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC score: 0.633
- Train ROC-AUC: 0.721
- Test ROC-AUC: 0.601
- Best hyperparameters {'C': 1e-15}
******************************************

------------------------------------------------------------
**** Best Conformation's ROC-AUC using docking scores ****
> Train best conf. ROC-AUC: 0.647 	> Median ROC-AUC: 0.598
> Test best conf. ROC-AUC: 0.639 	> Median ROC-AUC: 0.584
------------------------------------------------------------
CPU times: user 3.04 s, sys: 287 ms, total: 3.32 s
Wall time: 16.5 s


<h3 style='color: #F84122; padding: 0px;; margin: 0px'>GS: Radial Basis Function SVM</h3>
<b>Merged Libraries</b>

In [42]:
import numpy as np
np.geomspace(1e0, 1e4, 4)
np.geomspace(1e-4, 1e0, 3)

array([1.e-04, 1.e-02, 1.e+00])

In [43]:
%%time
from sklearn.svm import SVC
estimator = SVC(kernel = 'rbf', probability=True)

hyperparams = {'C': np.geomspace(1e0, 1e2, 3), 
               'gamma': np.geomspace(1e-4, 1e0, 3)}

# RANDOM Train test splitting
split_and_gs(X, y, estimator, hyperparams, 
             splitting='random', test_size=0.25, 
             scaffold_series=None)

No. of molecules in train set: 4674, with 225 actives.
No. of molecules in test set: 1559, with 75 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC score: 0.878
- Train ROC-AUC: 1.000
- Test ROC-AUC: 0.891
- Best hyperparameters {'C': 10.0, 'gamma': 0.01}
******************************************

------------------------------------------------------------
**** Best Conformation's ROC-AUC using docking scores ****
> Train best conf. ROC-AUC: 0.698 	> Median ROC-AUC: 0.616
> Test best conf. ROC-AUC: 0.664 	> Median ROC-AUC: 0.587
------------------------------------------------------------
CPU times: user 9.61 s, sys: 38.5 ms, total: 9.65 s
Wall time: 3min 6s


In [65]:
%%time
estimator = SVC(kernel = 'rbf', probability=True)

hyperparams = {'C': np.geomspace(1e0, 1e2, 3), 
               'gamma': np.geomspace(1e-4, 1e0, 3)}

# SCAFFOLD Train test splitting
split_and_gs(X, y, estimator, hyperparams, 
             splitting='scaffold', test_size=0.25, 
             scaffold_series=scaffold_series)

No. of molecules in train set: 4449, with 225 actives.
No. of molecules in test set: 1784, with 75 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC score: 0.684
- Train ROC-AUC: 0.997
- Test ROC-AUC: 0.718
- Best hyperparameters {'C': 1.0, 'gamma': 0.01}
******************************************

------------------------------------------------------------
**** Best Conformation's ROC-AUC using docking scores ****
> Train best conf. ROC-AUC: 0.647 	> Median ROC-AUC: 0.598
> Test best conf. ROC-AUC: 0.639 	> Median ROC-AUC: 0.584
------------------------------------------------------------
CPU times: user 10.7 s, sys: 152 ms, total: 10.8 s
Wall time: 2min 41s


<h3 style='color: #F84122; padding: 0px;; margin: 0px'>GS: Logistic Regression</h3>
<b>Merged Libraries</b>

In [45]:
%%time
from sklearn.linear_model import LogisticRegression
estimator = LogisticRegression()

hyperparams = {'C': np.geomspace(1e-4, 1e4, 3), 
               'penalty': ['l1', 'l2'], 
               'solver': ['lbfgs', 'liblinear']}

# RANDOM Train test splitting
split_and_gs(X, y, estimator, hyperparams, 
             splitting='random', test_size=0.25, 
             scaffold_series=None)

No. of molecules in train set: 4674, with 225 actives.
No. of molecules in test set: 1559, with 75 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC score: 0.823
- Train ROC-AUC: 0.884
- Test ROC-AUC: 0.858
- Best hyperparameters {'C': 1.0, 'penalty': 'l1', 'solver': 'liblinear'}
******************************************

------------------------------------------------------------
**** Best Conformation's ROC-AUC using docking scores ****
> Train best conf. ROC-AUC: 0.682 	> Median ROC-AUC: 0.609
> Test best conf. ROC-AUC: 0.706 	> Median ROC-AUC: 0.605
------------------------------------------------------------
CPU times: user 6.84 s, sys: 0 ns, total: 6.84 s
Wall time: 33.3 s


In [66]:
%%time
# from sklearn.linear_model import LogisticRegression
estimator = LogisticRegression()

hyperparams = {'C': np.geomspace(1e-4, 1e5, 4), 
               'penalty': ['l1', 'l2'], 
               'solver': ['lbfgs', 'liblinear']}

# SCAFFOLD Train test splitting
split_and_gs(X, y, estimator, hyperparams, 
             splitting='scaffold', test_size=0.25, 
             scaffold_series=scaffold_series)

No. of molecules in train set: 4449, with 225 actives.
No. of molecules in test set: 1784, with 75 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC score: 0.657
- Train ROC-AUC: 0.822
- Test ROC-AUC: 0.670
- Best hyperparameters {'C': 100.0, 'penalty': 'l1', 'solver': 'liblinear'}
******************************************

------------------------------------------------------------
**** Best Conformation's ROC-AUC using docking scores ****
> Train best conf. ROC-AUC: 0.647 	> Median ROC-AUC: 0.598
> Test best conf. ROC-AUC: 0.639 	> Median ROC-AUC: 0.584
------------------------------------------------------------
CPU times: user 6.65 s, sys: 154 ms, total: 6.81 s
Wall time: 29.2 s


<h3 style='color: #F84122; padding: 0px;; margin: 0px'>GS: Decision Tree Classifier</h3>
<b>Merged Libraries</b>

In [47]:
%%time
from sklearn.tree import DecisionTreeClassifier

estimator = DecisionTreeClassifier(splitter='best')
hyperparams = {'criterion': ['gini', 'entropy'], 
               'max_depth': [2, 3, 5],
               'min_samples_split': [0.2,  0.3],
               'min_samples_leaf': [0.02, 0.05, 0.1],
               'max_features': [None, 'sqrt', 'log2']}

# RANDOM Train test splitting
split_and_gs(X, y, estimator, hyperparams, 
             splitting='random', test_size=0.25, 
             scaffold_series=None)

No. of molecules in train set: 4674, with 225 actives.
No. of molecules in test set: 1559, with 75 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC score: 0.742
- Train ROC-AUC: 0.812
- Test ROC-AUC: 0.719
- Best hyperparameters {'criterion': 'entropy', 'max_depth': 5, 'max_features': None, 'min_samples_leaf': 0.05, 'min_samples_split': 0.2}
******************************************

------------------------------------------------------------
**** Best Conformation's ROC-AUC using docking scores ****
> Train best conf. ROC-AUC: 0.69 	> Median ROC-AUC: 0.622
> Test best conf. ROC-AUC: 0.676 	> Median ROC-AUC: 0.567
------------------------------------------------------------
CPU times: user 2.57 s, sys: 27.8 ms, total: 2.59 s
Wall time: 5.95 s


In [67]:
%%time
from sklearn.tree import DecisionTreeClassifier

estimator = DecisionTreeClassifier(splitter='best')
hyperparams = {'criterion': ['gini', 'entropy'], 
               'max_depth': [2, 3, 5],
               'min_samples_split': [0.2,  0.3],
               'min_samples_leaf': [0.02, 0.05, 0.1],
               'max_features': [None, 'sqrt', 'log2']}

# SCAFFOLD Train test splitting
split_and_gs(X, y, estimator, hyperparams, 
             splitting='scaffold', test_size=0.25, 
             scaffold_series=scaffold_series)

No. of molecules in train set: 4449, with 225 actives.
No. of molecules in test set: 1784, with 75 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC score: 0.642
- Train ROC-AUC: 0.659
- Test ROC-AUC: 0.597
- Best hyperparameters {'criterion': 'entropy', 'max_depth': 2, 'max_features': 'sqrt', 'min_samples_leaf': 0.1, 'min_samples_split': 0.2}
******************************************

------------------------------------------------------------
**** Best Conformation's ROC-AUC using docking scores ****
> Train best conf. ROC-AUC: 0.647 	> Median ROC-AUC: 0.598
> Test best conf. ROC-AUC: 0.639 	> Median ROC-AUC: 0.584
------------------------------------------------------------
CPU times: user 2.64 s, sys: 216 ms, total: 2.85 s
Wall time: 6.02 s


<h3 style='color: #F84122; padding: 0px;; margin: 0px'>GS: Bagging Classifier (k-NN as base estimator) </h3>
<b>Merged Libraries</b>

In [49]:
%%time
from sklearn.ensemble import BaggingClassifier

knn = KNeighborsClassifier(n_neighbors=125, p=1, 
                           weights='distance')

estimator = BaggingClassifier(base_estimator=knn, 
                              n_jobs=3, oob_score=True)

hyperparams = {'n_estimators': [300]}

# RANDOM Train test splitting
# split_and_gs(X, y, estimator, hyperparams, 
#              splitting='random', test_size=0.25, 
#              scaffold_series=None)

CPU times: user 50 µs, sys: 0 ns, total: 50 µs
Wall time: 56.7 µs


In [50]:
%%time
# from sklearn.ensemble import BaggingClassifier

knn = KNeighborsClassifier(n_neighbors=125, p=1, 
                           weights='distance')

estimator = BaggingClassifier(base_estimator=knn, 
                              n_jobs=3, oob_score=True)

hyperparams = {'n_estimators': [300]}

# SCAFFOLD Train test splitting
# split_and_gs(X, y, estimator, hyperparams, 
#              splitting='scaffold', test_size=0.25, 
#              scaffold_series=scaffold_series)

CPU times: user 43 µs, sys: 0 ns, total: 43 µs
Wall time: 51.5 µs


<h3 style='color: #F84122; padding: 0px;; margin: 0px'>GS: Random Forest </h3>
<b>Merged Libraries</b>

In [51]:
%%time
from sklearn.ensemble import RandomForestClassifier

estimator = RandomForestClassifier()

hyperparams = {'n_estimators': [300, 400],
               'max_depth': [2, 3],
               'min_samples_split': [0.1,  0.3],
               'min_samples_leaf': [0.02, 0.05],
               'max_features': ['sqrt']
            }

# RANDOM Train test splitting
split_and_gs(X, y, estimator, hyperparams, 
             splitting='random', test_size=0.25, 
             scaffold_series=None)

No. of molecules in train set: 4674, with 225 actives.
No. of molecules in test set: 1559, with 75 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC score: 0.750
- Train ROC-AUC: 0.792
- Test ROC-AUC: 0.696
- Best hyperparameters {'max_depth': 3, 'max_features': 'sqrt', 'min_samples_leaf': 0.05, 'min_samples_split': 0.1, 'n_estimators': 400}
******************************************

------------------------------------------------------------
**** Best Conformation's ROC-AUC using docking scores ****
> Train best conf. ROC-AUC: 0.69 	> Median ROC-AUC: 0.611
> Test best conf. ROC-AUC: 0.67 	> Median ROC-AUC: 0.597
------------------------------------------------------------
CPU times: user 3.57 s, sys: 196 ms, total: 3.76 s
Wall time: 39.6 s


In [68]:
%%time
from sklearn.ensemble import RandomForestClassifier

estimator = RandomForestClassifier()

hyperparams = {'n_estimators': [300, 400],
               'max_depth': [2, 3],
               'min_samples_split': [0.1,  0.3],
               'min_samples_leaf': [0.02, 0.05],
               'max_features': ['sqrt']
            }

# SCAFFOLD Train test splitting
split_and_gs(X, y, estimator, hyperparams, 
             splitting='scaffold', test_size=0.25, 
             scaffold_series=scaffold_series)

No. of molecules in train set: 4449, with 225 actives.
No. of molecules in test set: 1784, with 75 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC score: 0.660
- Train ROC-AUC: 0.758
- Test ROC-AUC: 0.611
- Best hyperparameters {'max_depth': 3, 'max_features': 'sqrt', 'min_samples_leaf': 0.05, 'min_samples_split': 0.1, 'n_estimators': 400}
******************************************

------------------------------------------------------------
**** Best Conformation's ROC-AUC using docking scores ****
> Train best conf. ROC-AUC: 0.647 	> Median ROC-AUC: 0.598
> Test best conf. ROC-AUC: 0.639 	> Median ROC-AUC: 0.584
------------------------------------------------------------
CPU times: user 3.72 s, sys: 96.8 ms, total: 3.81 s
Wall time: 37.9 s
